In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers


df = pd.read_csv("E:\Final year project\project\datapreprocessing\iot_combined.csv")


top_labels = {
    'PartOfAHorizontalPortScan': 1,
    'Benign': 0,
    'Okiru': 2,
    'DDoS': 3,
}

df = df[df['label'].isin(top_labels)]

balanced_df = pd.concat([df[df['label'] == label].sample(12500, replace=True) for label in top_labels])


balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)
balanced_df['label'] = balanced_df['label'].map(top_labels)

In [3]:
from tensorflow.keras.utils import to_categorical

# Extract features and labels
X = balanced_df.drop(columns=['label'])
y = balanced_df['label']

# Convert the labels to one-hot encoding
num_classes = 4
y_encoded = to_categorical(y, num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Reshape the data for CNN input
input_shape = (24, 1)
X_train_reshaped = X_train.values.reshape(X_train.shape[0], *input_shape)
X_test_reshaped = X_test.values.reshape(X_test.shape[0], *input_shape)

# Create and train the CNN model
cnn_model = keras.Sequential([
    layers.Conv1D(32, 3, activation='relu', input_shape=input_shape),
    layers.MaxPooling1D(2),
    layers.Conv1D(64, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_reshaped, y_train, epochs=10, validation_data=(X_test_reshaped, y_test))

# Extract features using the trained CNN model
feature_extractor = keras.Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
X_train_features = feature_extractor.predict(X_train_reshaped)
X_test_features = feature_extractor.predict(X_test_reshaped)

# Reshape the data for LSTM input
X_train_features = X_train_features.reshape(X_train_features.shape[0], X_train_features.shape[1], 1)
X_test_features = X_test_features.reshape(X_test_features.shape[0], X_test_features.shape[1], 1)

# Create and train the LSTM model
lstm_model = keras.Sequential([
    layers.LSTM(64, input_shape=(X_train_features.shape[1], 1)),
    layers.Dense(num_classes, activation='softmax')
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_features, y_train, epochs=10, validation_data=(X_test_features, y_test))

# Evaluate the LSTM model
loss, accuracy = lstm_model.evaluate(X_test_features, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/10
1250/1250 [==============================] - 5s 3ms/step - loss: 583.3860 - accuracy: 0.3849 - val_loss: 99.4382 - val_accuracy: 0.4116
Epoch 2/10
1250/1250 [==============================] - 3s 3ms/step - loss: 38.4765 - accuracy: 0.5231 - val_loss: 9.4881 - val_accuracy: 0.5674
Epoch 3/10
1250/1250 [==============================] - 3s 3ms/step - loss: 2.2051 - accuracy: 0.6536 - val_loss: 1.3414 - val_accuracy: 0.7048
Epoch 4/10
1250/1250 [==============================] - 4s 3ms/step - loss: 0.9690 - accuracy: 0.6564 - val_loss: 0.7423 - val_accuracy: 0.6763
Epoch 5/10
1250/1250 [==============================] - 3s 3ms/step - loss: 0.8836 - accuracy: 0.6589 - val_loss: 0.7758 - val_accuracy: 0.6478
Epoch 6/10
1250/1250 [==============================] - 4s 3ms/step - loss: 0.8120 - accuracy: 0.6597 - val_loss: 0.7346 - val_accuracy: 0.6762
Epoch 7/10
1250/1250 [==============================] - 4s 3ms/step - loss: 0.7807 - accuracy: 0.6611 - val_loss: 0.6841 - val_accur

In [6]:
from sklearn.metrics import confusion_matrix, classification_report


# Predictions on the test set
y_pred_probs = lstm_model.predict(X_test_features)

# Convert probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Generate classification report
report = classification_report(y_true, y_pred)
print("Classification Report:\n", report)

313/313 [==============================] - 4s 12ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.54      0.68      2557
           1       0.59      0.50      0.54      2418
           2       0.49      0.87      0.63      2523
           3       1.00      0.80      0.89      2502

    accuracy                           0.68     10000
   macro avg       0.75      0.68      0.68     10000
weighted avg       0.75      0.68      0.68     10000


using RMSPROP

In [3]:
from tensorflow.keras.utils import to_categorical

# Extract features and labels
X = balanced_df.drop(columns=['label'])
y = balanced_df['label']

# Convert the labels to one-hot encoding
num_classes = 4
y_encoded = to_categorical(y, num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Reshape the data for CNN input
input_shape = (24, 1)
X_train_reshaped = X_train.values.reshape(X_train.shape[0], *input_shape)
X_test_reshaped = X_test.values.reshape(X_test.shape[0], *input_shape)

# Create and train the CNN model
cnn_model = keras.Sequential([
    layers.Conv1D(32, 3, activation='relu', input_shape=input_shape),
    layers.MaxPooling1D(2),
    layers.Conv1D(64, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

cnn_model.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_reshaped, y_train, epochs=30, validation_data=(X_test_reshaped, y_test))

# Extract features using the trained CNN model
feature_extractor = keras.Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
X_train_features = feature_extractor.predict(X_train_reshaped)
X_test_features = feature_extractor.predict(X_test_reshaped)

# Reshape the data for LSTM input
X_train_features = X_train_features.reshape(X_train_features.shape[0], X_train_features.shape[1], 1)
X_test_features = X_test_features.reshape(X_test_features.shape[0], X_test_features.shape[1], 1)

# Create and train the LSTM model
lstm_model = keras.Sequential([
    layers.LSTM(64, input_shape=(X_train_features.shape[1], 1)),
    layers.Dense(num_classes, activation='softmax')
])

lstm_model.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_features, y_train, epochs=30, validation_data=(X_test_features, y_test))

# Evaluate the LSTM model
loss, accuracy = lstm_model.evaluate(X_test_features, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/30
1250/1250 [==============================] - 4s 3ms/step - loss: 717.4614 - accuracy: 0.4556 - val_loss: 0.8395 - val_accuracy: 0.5947
Epoch 2/30
1250/1250 [==============================] - 3s 2ms/step - loss: 13.6069 - accuracy: 0.6162 - val_loss: 233.7913 - val_accuracy: 0.2507
Epoch 3/30
1250/1250 [==============================] - 3s 3ms/step - loss: 2.9442 - accuracy: 0.6533 - val_loss: 0.7118 - val_accuracy: 0.5684
Epoch 4/30
1250/1250 [==============================] - 3s 3ms/step - loss: 21.4531 - accuracy: 0.6466 - val_loss: 0.6874 - val_accuracy: 0.6765
Epoch 5/30
1250/1250 [==============================] - 3s 3ms/step - loss: 56.7080 - accuracy: 0.5109 - val_loss: 0.9029 - val_accuracy: 0.6749
Epoch 6/30
1250/1250 [==============================] - 3s 2ms/step - loss: 3.5311 - accuracy: 0.6395 - val_loss: 0.7385 - val_accuracy: 0.6760
Epoch 7/30
1250/1250 [==============================] - 3s 2ms/step - loss: 2.7120 - accuracy: 0.6567 - val_loss: 0.6928 - val_ac

In [4]:
from tensorflow.keras.utils import to_categorical

# Extract features and labels
X = balanced_df.drop(columns=['label'])
y = balanced_df['label']

# Convert the labels to one-hot encoding
num_classes = 4
y_encoded = to_categorical(y, num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Reshape the data for CNN input
input_shape = (24, 1)
X_train_reshaped = X_train.values.reshape(X_train.shape[0], *input_shape)
X_test_reshaped = X_test.values.reshape(X_test.shape[0], *input_shape)

# Create and train the CNN model
cnn_model = keras.Sequential([
    layers.Conv1D(64, 3, activation='relu', input_shape=input_shape),
    layers.MaxPooling1D(2),
    layers.Conv1D(128, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

cnn_model.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_reshaped, y_train, epochs=30, validation_data=(X_test_reshaped, y_test))

# Extract features using the trained CNN model
feature_extractor = keras.Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
X_train_features = feature_extractor.predict(X_train_reshaped)
X_test_features = feature_extractor.predict(X_test_reshaped)

# Reshape the data for LSTM input
X_train_features = X_train_features.reshape(X_train_features.shape[0], X_train_features.shape[1], 1)
X_test_features = X_test_features.reshape(X_test_features.shape[0], X_test_features.shape[1], 1)

# Create and train the LSTM model
lstm_model = keras.Sequential([
    layers.LSTM(128, input_shape=(X_train_features.shape[1], 1)),
    layers.Dense(num_classes, activation='softmax')
])

lstm_model.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_features, y_train, epochs=30, validation_data=(X_test_features, y_test))

# Evaluate the LSTM model
loss, accuracy = lstm_model.evaluate(X_test_features, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/30
1250/1250 [==============================] - 5s 3ms/step - loss: 553.7194 - accuracy: 0.4062 - val_loss: 0.8459 - val_accuracy: 0.5653
Epoch 2/30
1250/1250 [==============================] - 4s 3ms/step - loss: 17.0318 - accuracy: 0.3368 - val_loss: 1.3798 - val_accuracy: 0.2615
Epoch 3/30
1250/1250 [==============================] - 4s 3ms/step - loss: 3.1034 - accuracy: 0.2646 - val_loss: 1.3373 - val_accuracy: 0.2851
Epoch 4/30
1250/1250 [==============================] - 4s 3ms/step - loss: 3.4274 - accuracy: 0.2600 - val_loss: 1.3488 - val_accuracy: 0.2886
Epoch 5/30
1250/1250 [==============================] - 4s 3ms/step - loss: 2.9956 - accuracy: 0.2622 - val_loss: 1.3745 - val_accuracy: 0.2578
Epoch 6/30
1250/1250 [==============================] - 4s 3ms/step - loss: 3.0823 - accuracy: 0.2893 - val_loss: 1.3581 - val_accuracy: 0.2694
Epoch 7/30
1250/1250 [==============================] - 4s 3ms/step - loss: 7.5302 - accuracy: 0.2967 - val_loss: 1.3720 - val_accura

regularization l2

In [5]:
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical

# Extract features and labels
X = balanced_df.drop(columns=['label'])
y = balanced_df['label']

# Convert the labels to one-hot encoding
num_classes = 4
y_encoded = to_categorical(y, num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Reshape the data for CNN input
input_shape = (24, 1)
X_train_reshaped = X_train.values.reshape(X_train.shape[0], *input_shape)
X_test_reshaped = X_test.values.reshape(X_test.shape[0], *input_shape)

# Create and train the CNN model with regularization
cnn_model = keras.Sequential([
    layers.Conv1D(32, 3, activation='relu', input_shape=input_shape, kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling1D(2),
    layers.Dropout(0.25),  # Adding dropout for regularization
    layers.Conv1D(64, 3, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.MaxPooling1D(2),
    layers.Dropout(0.25),  # Adding dropout for regularization
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(num_classes, activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_reshaped, y_train, epochs=10, validation_data=(X_test_reshaped, y_test))

# ... (previous code)

# Create and train the LSTM model with regularization
lstm_model = keras.Sequential([
    layers.LSTM(64, input_shape=(X_train_features.shape[1], 1), kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.25),  # Adding dropout for regularization
    layers.Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(0.01))
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_features, y_train, epochs=10, validation_data=(X_test_features, y_test))

# ... (previous code)

# Evaluate the LSTM model
loss, accuracy = lstm_model.evaluate(X_test_features, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Epoch 1/10
1250/1250 [==============================] - 6s 3ms/step - loss: 517.7504 - accuracy: 0.4139 - val_loss: 2.3625 - val_accuracy: 0.5740
Epoch 2/10
1250/1250 [==============================] - 4s 3ms/step - loss: 4.5400 - accuracy: 0.5066 - val_loss: 1.8054 - val_accuracy: 0.5492
Epoch 3/10
1250/1250 [==============================] - 4s 3ms/step - loss: 2.2684 - accuracy: 0.5156 - val_loss: 1.6543 - val_accuracy: 0.4727
Epoch 4/10
1250/1250 [==============================] - 4s 3ms/step - loss: 2.2239 - accuracy: 0.5222 - val_loss: 1.5921 - val_accuracy: 0.5054
Epoch 5/10
1250/1250 [==============================] - 4s 3ms/step - loss: 2.0831 - accuracy: 0.5184 - val_loss: 1.5529 - val_accuracy: 0.4717
Epoch 6/10
1250/1250 [==============================] - 4s 3ms/step - loss: 1.7365 - accuracy: 0.5263 - val_loss: 1.5238 - val_accuracy: 0.5494
Epoch 7/10
1250/1250 [==============================] - 4s 3ms/step - loss: 1.9953 - accuracy: 0.5258 - val_loss: 1.5048 - val_accurac

In [2]:
from tensorflow.keras.utils import to_categorical

# Extract features and labels
X = balanced_df.drop(columns=['label'])
y = balanced_df['label']

# Convert the labels to one-hot encoding
num_classes = 4
y_encoded = to_categorical(y, num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Reshape the data for CNN input
input_shape = (24, 1)
X_train_reshaped = X_train.values.reshape(X_train.shape[0], *input_shape)
X_test_reshaped = X_test.values.reshape(X_test.shape[0], *input_shape)

cnn_model = keras.Sequential([
    layers.Conv1D(64, 3, activation='relu', input_shape=input_shape),
    layers.BatchNormalization(),
    layers.MaxPooling1D(2),
    layers.Conv1D(128, 3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(256),
    layers.LeakyReLU(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])


cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_reshaped, y_train, epochs=10, validation_data=(X_test_reshaped, y_test))

# Extract features using the trained CNN model
feature_extractor = keras.Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
X_train_features = feature_extractor.predict(X_train_reshaped)
X_test_features = feature_extractor.predict(X_test_reshaped)

# Reshape the data for LSTM input
X_train_features = X_train_features.reshape(X_train_features.shape[0], X_train_features.shape[1], 1)
X_test_features = X_test_features.reshape(X_test_features.shape[0], X_test_features.shape[1], 1)

lstm_model = keras.Sequential([
    layers.LSTM(128, return_sequences=True, input_shape=(X_train_features.shape[1], 1)),
    layers.LSTM(64),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_features, y_train, epochs=10, validation_data=(X_test_features, y_test))

# Evaluate the LSTM model
loss, accuracy = lstm_model.evaluate(X_test_features, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/10
1250/1250 [==============================] - 21s 14ms/step - loss: 0.7711 - accuracy: 0.6586 - val_loss: 1.7977 - val_accuracy: 0.4165
Epoch 2/10
1250/1250 [==============================] - 18s 14ms/step - loss: 0.6815 - accuracy: 0.6873 - val_loss: 1.5427 - val_accuracy: 0.4623
Epoch 3/10
1250/1250 [==============================] - 16s 13ms/step - loss: 0.6581 - accuracy: 0.6956 - val_loss: 1.1951 - val_accuracy: 0.5028
Epoch 4/10
1250/1250 [==============================] - 17s 13ms/step - loss: 0.6501 - accuracy: 0.6994 - val_loss: 0.9589 - val_accuracy: 0.5533
Epoch 5/10
1250/1250 [==============================] - 18s 14ms/step - loss: 0.6436 - accuracy: 0.6996 - val_loss: 1.0151 - val_accuracy: 0.5500
Epoch 6/10
1250/1250 [==============================] - 15s 12ms/step - loss: 0.6425 - accuracy: 0.6998 - val_loss: 0.7567 - val_accuracy: 0.6365
Epoch 7/10
1250/1250 [==============================] - 16s 13ms/step - loss: 0.6339 - accuracy: 0.7029 - val_loss: 0.7195 -

In [3]:
from tensorflow.keras.utils import to_categorical

# Extract features and labels
X = balanced_df.drop(columns=['label'])
y = balanced_df['label']

# Convert the labels to one-hot encoding
num_classes = 4
y_encoded = to_categorical(y, num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Reshape the data for CNN input
input_shape = (24, 1)
X_train_reshaped = X_train.values.reshape(X_train.shape[0], *input_shape)
X_test_reshaped = X_test.values.reshape(X_test.shape[0], *input_shape)

cnn_model = keras.Sequential([
    layers.Conv1D(64, 3, activation='relu', input_shape=input_shape),
    layers.BatchNormalization(),
    layers.MaxPooling1D(2),
    layers.Conv1D(128, 3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(256),
    layers.LeakyReLU(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])


cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_reshaped, y_train, epochs=10, validation_data=(X_test_reshaped, y_test))

# Extract features using the trained CNN model
feature_extractor = keras.Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
X_train_features = feature_extractor.predict(X_train_reshaped)
X_test_features = feature_extractor.predict(X_test_reshaped)

# Reshape the data for LSTM input
X_train_features = X_train_features.reshape(X_train_features.shape[0], X_train_features.shape[1], 1)
X_test_features = X_test_features.reshape(X_test_features.shape[0], X_test_features.shape[1], 1)

lstm_model = keras.Sequential([
    layers.LSTM(256, return_sequences=True, input_shape=(X_train_features.shape[1], 1)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.LSTM(128),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_features, y_train, epochs=10, validation_data=(X_test_features, y_test))

# Evaluate the LSTM model
loss, accuracy = lstm_model.evaluate(X_test_features, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Epoch 1/10
1250/1250 [==============================] - 8s 5ms/step - loss: 0.7804 - accuracy: 0.6551 - val_loss: 1.2008 - val_accuracy: 0.5246
Epoch 2/10
1250/1250 [==============================] - 6s 4ms/step - loss: 0.6781 - accuracy: 0.6916 - val_loss: 1.0622 - val_accuracy: 0.5321
Epoch 3/10
1250/1250 [==============================] - 6s 4ms/step - loss: 0.6551 - accuracy: 0.6981 - val_loss: 57.9087 - val_accuracy: 0.2524
Epoch 4/10
1250/1250 [==============================] - 5s 4ms/step - loss: 0.6501 - accuracy: 0.6976 - val_loss: 0.8027 - val_accuracy: 0.5942
Epoch 5/10
1250/1250 [==============================] - 6s 5ms/step - loss: 0.6446 - accuracy: 0.7027 - val_loss: 0.8561 - val_accuracy: 0.5571
Epoch 6/10
1250/1250 [==============================] - 6s 4ms/step - loss: 0.6383 - accuracy: 0.7031 - val_loss: 1.0654 - val_accuracy: 0.6316
Epoch 7/10
1250/1250 [==============================] - 5s 4ms/step - loss: 0.6359 - accuracy: 0.7038 - val_loss: 0.7880 - val_accuracy

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, BatchNormalization, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow.keras as keras


# Extract features and labels
X = balanced_df.drop(columns=['label'])
y = balanced_df['label']

# Convert the labels to one-hot encoding
num_classes = 4
y_encoded = to_categorical(y, num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the data for CNN input
input_shape = (X_train.shape[1], 1)  # Adjusted input shape
X_train_reshaped = X_train.reshape(X_train.shape[0], *input_shape)
X_test_reshaped = X_test.reshape(X_test.shape[0], *input_shape)

# CNN Model
cnn_model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

cnn_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_reshaped, y_train, epochs=15, validation_data=(X_test_reshaped, y_test))

# Extract features using the trained CNN model
feature_extractor = keras.Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
X_train_features = feature_extractor.predict(X_train_reshaped)
X_test_features = feature_extractor.predict(X_test_reshaped)

# LSTM Model
lstm_model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train_features.shape[1], 1)),
    BatchNormalization(),
    Dropout(0.5),
    LSTM(64),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

lstm_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_features, y_train, epochs=15, validation_data=(X_test_features, y_test))

# Evaluate the LSTM model
loss, accuracy = lstm_model.evaluate(X_test_features, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Epoch 1/15
1250/1250 [==============================] - 25s 18ms/step - loss: 0.6566 - accuracy: 0.7018 - val_loss: 0.5881 - val_accuracy: 0.7220
Epoch 2/15
1250/1250 [==============================] - 23s 18ms/step - loss: 0.6265 - accuracy: 0.7147 - val_loss: 0.6241 - val_accuracy: 0.7277
Epoch 3/15
1250/1250 [==============================] - 20s 16ms/step - loss: 0.6148 - accuracy: 0.7176 - val_loss: 0.6218 - val_accuracy: 0.7173
Epoch 4/15
1250/1250 [==============================] - 20s 16ms/step - loss: 0.6110 - accuracy: 0.7175 - val_loss: 0.6003 - val_accuracy: 0.7283
Epoch 5/15
1250/1250 [==============================] - 21s 17ms/step - loss: 0.6085 - accuracy: 0.7192 - val_loss: 0.6174 - val_accuracy: 0.7218
Epoch 6/15
1250/1250 [==============================] - 21s 17ms/step - loss: 0.6079 - accuracy: 0.7195 - val_loss: 0.6386 - val_accuracy: 0.7299
Epoch 7/15
1250/1250 [==============================] - 21s 17ms/step - loss: 0.6038 - accuracy: 0.7200 - val_loss: 0.6029 -

In [6]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.metrics import accuracy_score

# Define a simple learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch % 5 == 0 and epoch > 0:
        return lr * 0.9  # Adjust the multiplication factor as needed
    return lr

# ...

# CNN Model
cnn_model = Sequential([
    layers.Conv1D(64, 3, activation='relu', input_shape=input_shape),
    layers.BatchNormalization(),
    layers.MaxPooling1D(2),
    layers.Conv1D(128, 3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

cnn_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Implement learning rate scheduling
lr_callback = LearningRateScheduler(lr_scheduler)

# Train the CNN model with learning rate scheduling
cnn_model.fit(X_train_reshaped, y_train, epochs=15, validation_data=(X_test_reshaped, y_test), callbacks=[lr_callback])

# ...

# LSTM Model
lstm_model = Sequential([
    layers.LSTM(128, return_sequences=True, input_shape=(X_train_features.shape[1], 1)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.LSTM(64),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

lstm_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Implement learning rate scheduling
lr_callback_lstm = LearningRateScheduler(lr_scheduler)

# Train the LSTM model with learning rate scheduling
lstm_model.fit(X_train_features, y_train, epochs=15, validation_data=(X_test_features, y_test), callbacks=[lr_callback_lstm])

# ...

# Evaluate the LSTM model
loss, accuracy = lstm_model.evaluate(X_test_features, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Epoch 1/15
1250/1250 [==============================] - 9s 5ms/step - loss: 0.6587 - accuracy: 0.7019 - val_loss: 0.6140 - val_accuracy: 0.7158 - lr: 0.0010
Epoch 2/15
1250/1250 [==============================] - 6s 5ms/step - loss: 0.6238 - accuracy: 0.7168 - val_loss: 0.6162 - val_accuracy: 0.7175 - lr: 0.0010
Epoch 3/15
1250/1250 [==============================] - 6s 5ms/step - loss: 0.6173 - accuracy: 0.7178 - val_loss: 0.6274 - val_accuracy: 0.7128 - lr: 0.0010
Epoch 4/15
1250/1250 [==============================] - 6s 5ms/step - loss: 0.6103 - accuracy: 0.7181 - val_loss: 0.5913 - val_accuracy: 0.7187 - lr: 0.0010
Epoch 5/15
1250/1250 [==============================] - 6s 5ms/step - loss: 0.6079 - accuracy: 0.7204 - val_loss: 0.5895 - val_accuracy: 0.7274 - lr: 0.0010
Epoch 6/15
1250/1250 [==============================] - 6s 5ms/step - loss: 0.6038 - accuracy: 0.7204 - val_loss: 0.6014 - val_accuracy: 0.7286 - lr: 9.0000e-04
Epoch 7/15
1250/1250 [==============================] 